In [2]:
import os
import re

In [4]:
rootdir = "/Users/abelmathew/Desktop/Semester 2/VDA2/Project/Files"
regex=re.compile('(.*rpt$)|(.*rar$)')
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if ("timing" in file)and("check" not in file):
            print(file)

mac_wi_sm.timing.rpt
mac_wi_sm.timing_derated_setup.rpt
mac_wi_sm.timing_final_place.rpt
gba_period8_fir.timing_hold.rpt
gba_period4_fir.timing_setup.rpt
pba_period4_fir.timing_setup.rpt
gba_period4_fir.timing_hold.rpt
pba_period4_fir.timing_hold.rpt
pba_period8_fir.timing_setup.rpt
gba_period8_fir.timing_setup.rpt
pba_period8_fir.timing_hold.rpt


In [144]:
def fileParsing(fileName,rootdir,designDirectoryName):
    pathgroupdict={}
    Point=[]
    Fanout=[]
    Cap=[]
    Tran=[]
    Incr=[]
    Path=[]
    HVTCount=0
    LVTCount=0
    RVTCount=0
#     fileName="pba_period8_fir.timing_setup.rpt"
    initialCase=1
    fileNamePath=rootdir+"/"+designDirectoryName+"/"+fileName
#     print(fileNamePath)
    timingFile=open(fileNamePath,'r')

    for line in timingFile.readlines():
        if line.strip():
    #         print(line)
            match initialCase:
                case 1 : 
                    if "-path_type" in line:
                        pathType=line.split(" ")[-1].strip()
                        HVTCount=0
                        LVTCount=0
                        RVTCount=0
                        initialCase=2
                        continue
                case 2 :
                    if "-delay_type" in line: 
                        delayType=line.split(" ")[-1].strip()
                        initialCase=3
                        continue
                case 3 :
                    if "-max_path" in line:
                        maxPath=line.split(" ")[-1].strip()
        #                 print(maxPath)
                        initialCase=4
                        continue
                case 4 :
                    if "-pba_mode" in line:
                        analysisMode='PBA'
                        initialCase=5
                        continue

                    elif "Design :" in line:
                        designName=line.split(" ")[-1].strip()
                        analysisMode='GBA'
                        initialCase=6
                        continue
                case 5 :
                    if "Design :" in line:
                        designName=line.split(" ")[-1].strip()
                        initialCase=6
                        continue

                case 6 :
                    if "Startpoint" in line:
                        startPoint=line.split(":")[-1][1:].strip()
                        initialCase=7
                        continue
                case 7 :
                    if "Endpoint" in line:
                        endPoint=line.split(":")[-1][1:].strip()
        #                 print(endPoint)
                        initialCase=8
                        continue
                case 8 : 
                    if "Path Group:" in line:
                        pathGroup=line.split(":")[-1][1:].strip()
        #                 print(pathGroup)
                        initialCase=9
                        continue
                case 9 :
                    if ("Point" in line)and("Fanout" in line)and("Cap" in line)and("Trans" in line)and("Incr" in line)and("Path" in line):
    #                     print("it hit here")
                        initialCase=10
                        continue
                case 10 :
                    if "----------" in line:
    #                     print("here also")
    #                     print(line)
                        initialCase=11
                        continue
                case 11 :
                    if "---------" in line:
                        initialCase=12
                        continue
                    else:
    #                     print(line)
                        templine=line.split("  ")
                        tempvalue=line.split("  ")[-1]
                        Point.append(templine[1])
                        if "HVT" in templine[1]:
                            HVTCount=HVTCount+1
                        if "LVT" in templine[1]:
                            LVTCount=LVTCount+1
                        if ("RVT" in templine[1])or("SVT" in templine[1]):
                            RVTCount=RVTCount+1
                        
                        if "(net)" in line:
    #                         print(tempvalue.split())
                            Fanout.append(tempvalue.split()[0])
                            Cap.append(tempvalue.split()[1])
                            Tran.append("")
                            Incr.append("")
                            Path.append("")
                        else:
                            Fanout.append("")
                            Cap.append("")
                            templist=tempvalue.replace("r","").replace("f","").replace("&","").replace("*","").replace("\n","").split(" ")
                            templist=[x for x in templist if x]

    #                         print(len(templist))
    #                         print(tempvalue.replace("r","").replace("f","").replace("&","").replace("*","").replace("\n","").split(" "))
                            if(len(templist)==3):
                                Tran.append(templist[-3])
                                Incr.append(templist[-2])
                                Path.append(templist[-1])
                            elif(len(templist)==2):
                                Tran.append("")
                                Incr.append(templist[-2])
                                Path.append(templist[-1])   
                            elif(len(templist)==1):
                                Tran.append("")
                                Incr.append("")
                                Path.append(templist[-1])
    #                     print("\n")
                case 12 :
                    if "required time" in line:
                        requiredTime=line.split(" ")[-1].strip()
                        initialCase=13
                        continue
                case 13 :
                    if "arrival time" in line:
                        arrivalTime=line.split("  ")[-1].strip()
                        initialCase=14
                        continue
                case 14 :
                    if "slack" in line:
                        slack=line.split("  ")[-1].strip()
                        if "MET" in line:
                            violated=False
                        elif "VIOLATED" in line:
                            violated=True
                        initialCase=15
                        continue
                case 15 :
                    timingData={"Path Element":Point,"Fanout":Fanout,"Capacitance":Cap,"Increment":Incr,"Path":Path}
                    df=pd.DataFrame(timingData)
                    
                    trialdict={"pathType":pathType,"delayType":delayType,"maxPath":maxPath,"designName":designName,"analysisMode":analysisMode,"startPoint":startPoint,"endPoint":endPoint,"pathGroup":pathGroup,"requiredTime":requiredTime,"arrivalTime":arrivalTime,"slack":slack,"violated":violated,"PathDetails":df,"HVTs":HVTCount,"LVTs":LVTCount,"RVTs":RVTCount}
                    pathgroupdict[pathGroup]=trialdict
#                     print(pathgroupdict)
    #                 print(pathGroup)



                #   reset all the values before reading again         
                    Point=[]
                    Fanout=[]
                    Cap=[]
                    Tran=[]
                    Incr=[]
                    Path=[] 
                    initialCase=1
                    continue
    return pathgroupdict
# Done after each individual file run
# Data Structure Organization
# if analysisMode=="PBA":
#     if delayType=="max":
#         suhdictpba["setupAnalysis"]=pathgroupdict

#     elif delayType=="min":
#         suhdictpba["holdAnalysis"]=pathgroupdict
#     PandGdict["PBA"]=suhdictpba

# elif analysisMode=="GBA":
#     if delayType=="max":
#         suhdictgba["setupAnalysis"]=pathgroupdict

#     elif delayType=="min":
#         suhdictgba["holdAnalysis"]=pathgroupdict
#     PandGdict["GBA"]=suhdictgba
# finaldict[designName]=PandGdict
#  filename : dict1
# dict1 = {pba : dict2 ,gba : dict3}
# dict3 = {setupGBA : dict11 , holdGBA : dict15}
# dict2 = {setupPBA : dict4 , holdPBA : dict5}
# trialdict - {pathGroup : dict6} // output: somedict reg2reg :somedict that dict is equivalent to the trialdict




In [52]:
fileName="pba_period8_fir.timing_setup.rpt"
rootdir = "/Users/abelmathew/Desktop/Semester 2/VDA2/Project/Files"
designdirName="FIR"
fp_phase1_dict=fileParsing(fileName,rootdir,designdirName)


In [151]:
finaldict={}
rootdir = "/Users/abelmathew/Desktop/Semester 2/VDA2/Project/Files"
for folders in os.listdir(rootdir):
    a={}
    dummydict_pba={}
    dummydict_gba={}
    dummydict_design={}
#     print(dummydict_design)
    if(os.path.exists(os.path.join(rootdir,folders))):
        if not folders.startswith("."):       
            for files in os.listdir(os.path.join(rootdir,folders)):
                if not files.startswith("."):
                    designdirName=folders
#                     print(designdirName)
                    a[files]=fileParsing(files,rootdir,designdirName)
#     print(a)
            for key in a:
                if("gba" in key):
                    if("setup" in key):
                        dummydict_gba["Setup"]=a[key]
                    elif("hold" in key):
                        dummydict_gba["Hold"]=a[key]

                if("pba" in key):
                    if("setup" in key):
                        dummydict_pba["Setup"]=a[key]
                    elif("hold" in key):
                        dummydict_pba["Hold"]=a[key]
#     print(designdirName,dummydict_gba)
            dummydict_design["GBA"]=dummydict_gba
            dummydict_design["PBA"]=dummydict_pba
            finaldict[designdirName]=dummydict_design
            print(designdirName,dummydict_design)

            
    #append to final

FIR_Period8 {'GBA': {'Hold': {'comb': {'pathType': 'full', 'delayType': 'min', 'maxPath': '1', 'designName': 'full_chip_fir', 'analysisMode': 'GBA', 'startPoint': 'test_se (input port clocked by CLK)', 'endPoint': 'test_so (output port clocked by CLK)', 'pathGroup': 'comb', 'requiredTime': '0.9792100191', 'arrivalTime': '-3.9092352390', 'slack': '2.9300253391', 'violated': False, 'PathDetails':                             Path Element Fanout      Capacitance  \
0                  clock CLK (rise edge)                           
1            clock network delay (ideal)                           
2                   input external delay                           
3                           test_se (in)                           
4                          test_se (net)      1  2535.5549316406   
5       I1025_W_test_se/PADIO (I1025_EW)                           
6        I1025_W_test_se/DOUT (I1025_EW)                           
7                  hvoHier_test_se (net)     35   134.0696

In [123]:
dummydict_pba={}
dummydict_gba={}
for key in a:
    if("gba" in key):
        if("setup" in key):
            dummydict_gba["Setup"]=a[key]
        elif("hold" in key):
            dummydict_gba["Hold"]=a[key]
            
    if("pba" in key):
        if("setup" in key):
            dummydict_pba["Setup"]=a[key]
        elif("hold" in key):
            dummydict_pba["Hold"]=a[key]

pba_period4_fir.timing_setup.rpt
pba_period4_fir.timing_hold.rpt


In [150]:
finaldict.keys()

dict_keys(['FIR_Period8', 'FIR_Period4'])

In [24]:
fp_phase1_dict

{'comb': {'pathType': 'full',
  'delayType': 'max',
  'maxPath': '1',
  'designName': 'full_chip_fir',
  'analysisMode': 'PBA',
  'startPoint': 'test_se (input port clocked by CLK)',
  'endPoint': 'test_so (output port clocked by CLK)',
  'pathGroup': 'comb',
  'requiredTime': '9.0164403915',
  'arrivalTime': '-4.3364648819',
  'slack': '4.6799750328',
  'violated': False,
  'PathDetails':                             Path Element Fanout      Capacitance  \
  0                  clock CLK (rise edge)                           
  1            clock network delay (ideal)                           
  2                   input external delay                           
  3                           test_se (in)                           
  4                          test_se (net)      1  2510.4511718750   
  5       I1025_W_test_se/PADIO (I1025_EW)                           
  6        I1025_W_test_se/DOUT (I1025_EW)                           
  7                  hvoHier_test_se (net)     35

In [234]:
Point=[]
Incr=[]
Path=[]

initialCase=1
fileName="mac_wi_sm.timing.rpt"
fileNamePath=rootdir+"/"+fileName
timingFile=open(fileNamePath,'r')
for line in timingFile.readlines():
    if line.strip():
#         print(line)
        match initialCase:
            case 1 : 
                if "-path_type" in line:
                    pathType=line.split(" ")[-1].strip()
                    initialCase=2
                    continue
            case 2 :
                if "-delay_type" in line: 
                    delayType=line.split(" ")[-1].strip()
                    initialCase=3
                    continue
            case 3 :
                if "-max_path" in line:
                    maxPath=line.split(" ")[-1].strip()
    #                 print(maxPath)
                    initialCase=4
                    continue
            case 4 :
                if "Design :" in line:
                    designName=line.split(" ")[-1].strip()
                    initialCase=5
                    continue
            case 5 :
                if "Startpoint" in line:
                    startPoint=line.split(":")[-1][1:].strip()
                    initialCase=6
                    continue
            case 6 :
                if "Endpoint" in line:
                    endPoint=line.split(":")[-1][1:].strip()
    #                 print(endPoint)
                    initialCase=7
                    continue
            case 7 : 
                if "Path Group:" in line:
                    pathGroup=line.split(":")[-1][1:].strip()
    #                 print(pathGroup)
                    initialCase=8
                    continue
            case 8 : 
                if ("Point" in line)and("Incr" in line)and("Path" in line):
                    initialCase=9
                    continue
            case 9 :
                if "----------" in line:
#                     print(line)
                    initialCase=10
                    continue
            case 10 : 
                
                if "---------" in line:
                    initialCase=11
                    continue
                else:
                    templine=line.split("    ")
#                     print(templine)
                    if("\n" in templine[0]):
                        Point.append(templine[0].strip())
                        initialCase=12
                        continue
                    else:
                        Point.append(templine[0].strip())
                        Incr.append(templine[-2].strip())
#                         print(Incr)
                        Path.append(templine[-1].strip())
#                     print(line)
            case 11 :
#                 print(line)
                if "required time" in line:
                    requiredTime=line.split(" ")[-1].strip()
                    initialCase=13
                    continue
                    
        
            case 12 :
                templineinter=line.split("    ")
#                 print(templineinter)
                Incr.append(templineinter[-2].strip())
                Path.append(templineinter[-1].strip().replace("~",""))
                initialCase=10
                continue
            
            case 13 : 
                print(line)
                if "arrival time" in line:
                    arrivalTime=line.split("  ")[-1].strip()
                    initialCase=14
                    continue
            case 14 :
                if "slack" in line:
                    slack=line.split("  ")[-1].strip()
                if "v"
                    initialCase=15
                    
                

SyntaxError: expected ':' (3643867847.py, line 103)

In [207]:
remove_char=["~","f","r"]
Path1=[]
for s in Path:
    for cha in remove_char:
        s=s.replace(cha,"")
    Path1.append(s.strip()) 

In [208]:
import pandas as pd
timingData={"Path Element":Point,"Increment":Incr,"Path":Path1}
df=pd.DataFrame(timingData)
trialdict={"pathType":pathType,"delayType":delayType,"maxPath":maxPath,"designName":designName,"startPoint":startPoint,"endPoint":endPoint,"pathGroup":pathGroup,"DataF":df}

In [33]:
rootdir = "/Users/abelmathew/Desktop/Semester 2/VDA2/Project/Files"
preProcessing(rootdir)

mac_wi_sm.timing.rpt
mac_wi_sm.timing_derated_setup.rpt
mac_wi_sm.timing_final_place.rpt
